In [48]:
import numpy as np
from qiskit import *
%matplotlib inline
from qiskit.visualization import plot_histogram
from qiskit.providers.ibmq import least_busy
from qiskit.tools.monitor import job_monitor
from qiskit import IBMQ
from qiskit import BasicAer
from math import pi
import random
from bayes_opt import BayesianOptimization



## Single Unit QAOA Circuit

Here I develop a single unit of the QAOA circuit. The unit requires its own angles $\gamma$ and $\beta$, as well as the global adjacency matrix and edge weights.

### Driver Circuit

In [3]:
def driver_circuit(qregister, beta):
    """
    input: qregister QuantumRegister object, arbitrary # of qubits
            beta, specific rotation angle in radians in [0, pi]
    output: QuantumCircuit object, driver
    """
    driver = QuantumCircuit(qregister)
    driver.rx(beta, qregister)
    return driver

In [4]:
q = QuantumRegister(3)

In [8]:
driver_circuit(q, pi/2).draw()

### Cost Circuit

In [23]:
def cost_circuit(qregister, gamma, adjacency_matrix, edge_weights):
    """
    input: - qregister QuantumRegister object, arbitrary # of qubits
           - gamma, specific rotation angle in radians [0, 2pi]
           - adjacency_matrix, list of lists where ij = 1 if qubits are connected by directed edge
           - edge_weights, dictionary {(i, j):w_ij} mapping directed edge ij to weight w_ij
           
    output: QuantumCircuit object, cost unitary
    """
    
    # initialize circuit, and qubit count
    N = len(adjacency_matrix)
    cost = QuantumCircuit(qregister)
    
    # construct unit cost circuit, from commutivity
    for i in range(N):
        for j in range(N):
            if adjacency_matrix[i][j] == 1 and i!= j:
                
                # basic cost unit
                cost.cx(qregister[i], qregister[j])
                cost.rz(gamma * edge_weights[(i, j)], qregister[j])
                cost.cx(qregister[i], qregister[j])
                
    return cost
    

In [24]:
q = QuantumRegister(3)

In [25]:
a = [[1, 1, 1], [1, 1, 1], [1, 1, 1]]
w = np.zeros((3, 3))
for i in range(len(w)):
    for j in range(len(w)):
        w[i, j] = random.random()

In [27]:
cost_circuit(q, pi/4, a, w).draw()

### Single unit

In [28]:
def unit_circuit(qregister, adjacency_matrix, edge_weights, gamma, beta):
    return cost_circuit(qregister, gamma, adjacency_matrix, edge_weights) + driver_circuit(qregister, beta)

In [31]:
unit_circuit(q, a, w, pi/8, pi/4).draw();

## QAOA Circuit

In [33]:
def QAOA(adjacency_matrix, edge_weights, gamma_list, beta_list):
    """
    input: - adjacency_matrix, adjacency_matrix, list of lists where ij = 1 if qubits are connected by directed edge
           - edge_weights, dictionary {(i, j):w_ij} mapping directed edge ij to weight w_ij
           - gamma_list, list of cost angles
           - beta_list, list of beta angles
           
    output: QuantumCircuit object, result of appending multiple circuits for various different angles
            length of angles lists determines depth of circuit in unit_circuits
    """
    
    # initialize register and circuit
    N = len(adjacency_matrix) 
    qregister = QuantumRegister(N)
    circuit = QuantumCircuit(q)
    
    for gamma, beta in zip(gamma_list, beta_list):
        circuit += unit_circuit(qregister, adjacency_matrix, edge_weights, gamma, beta)
        
    return circuit

In [39]:
QAOA(a, w, [pi/4], [pi/6]).draw();

## Calling the QC

In [49]:
IBMQ.load_accounts(hub=None)

In [50]:
print("Available backends:")
IBMQ.backends()

Available backends:


[<IBMQBackend('ibmqx4') from IBMQ()>,
 <IBMQBackend('ibmqx2') from IBMQ()>,
 <IBMQBackend('ibmq_16_melbourne') from IBMQ()>,
 <IBMQSimulator('ibmq_qasm_simulator') from IBMQ()>]

In [51]:
from qiskit.providers.ibmq import least_busy

large_enough_devices = IBMQ.backends(filters=lambda x: x.configuration().n_qubits < 10 and
                                                       not x.configuration().simulator)
backend = least_busy(large_enough_devices)
print("The best backend is " + backend.name())

The best backend is ibmqx4


In [52]:
# construct adjacency matrix of specific layout
adjacency = [[0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 0], [0, 0, 1, 0, 1], [0, 0, 1, 0, 0]]

In [56]:
# randomize edge weights
wehts = [[random.random() for  i in range(len(adjacency))] for j in range(len(adjacency))]

## Bayesian Optimization

Here I use the Bayesian optimization package to optimize over the choice of gamma and beta. The depth, in the number of driving cycles p = 1, is assumed constant.

In [41]:
def optimizing_function(gamma, beta):
    
    

In [46]:
def optimize(adjacency_matrix, edge_weights):
    
    pbounds = {'gamma':(0, 2*pi), 'beta':(0,  pi)}
        
    optimizer = BayesianOptimization(
        f=optimizing_function,
        pbounds=pbounds,
        random_state=1,
    )
    
    optimizer.maximize(
    init_points=5,
    n_iter=25,
    )
    
    return
    
    
    